In [1]:
import pandas as pd
import  numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

In [2]:
df_train = pd.read_csv("mnist_train.csv")
df_test = pd.read_csv("mnist_test.csv")

In [3]:
df_train.head()

# 784 features and 1 label
# 1x1 is pixel0
# 1x2 is pixel1
# 1x3 is pixel2

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_test = df_test.drop('label',axis=1)

In [5]:
df_test.head()

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_train.shape

(60000, 785)

In [7]:
df_test.shape

(10000, 784)

In [8]:
df_features = df_train.iloc[:,1:785] #indepen
df_labels = df_train.iloc[:,0] #depen

x_test = df_test.iloc[:,0:784]

print(x_test.shape)

(10000, 784)


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_cv,y_train,y_cv = train_test_split(df_features,df_labels,test_size=0.2,random_state=1212)

# x_cv - is the validation data

In [10]:
x_train.shape #(48000, 784)
y_train.shape #(48000,)
x_cv.shape    # (12000,784)
y_cv.shape    # (12000,

(12000,)

In [11]:
x_train = x_train.values.reshape(48000, 784)
x_cv = x_cv.values.reshape(12000, 784)
x_test = x_test.values.reshape(10000, 784)


In [12]:
print((min(x_train[1]) , max(x_train[1])))
# min pixel value and max , grey scale img and value of this im are from 0 to 255

(0, 255)


In [13]:
# feature normalization
x_train = x_train.astype('float32')
x_cv = x_cv.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_cv /= 255
x_test /= 255


In [14]:
# convetr labels to one hot encoding
num_digits = 10
y_train = keras.utils.to_categorical(y_train,num_digits)
y_cv = keras.utils.to_categorical(y_cv,num_digits)

In [15]:
y_train[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [16]:
y_train[1]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [17]:
n_input = 784
n_hiddien_1 = 300
n_hiddien_2 = 100
n_hiddien_3 = 100
n_hiddien_4 = 200
num_digits = 10

In [18]:
# Model1

inp = Input(shape=(784,))
x =  Dense(n_hiddien_1 , activation='relu',name = 'Hidden_Layer_1')(inp)
x =  Dense(n_hiddien_2 , activation='relu',name = 'Hidden_Layer_2')(x)
x =  Dense(n_hiddien_3 , activation='relu',name = 'Hidden_Layer_3')(x)
x =  Dense(n_hiddien_4 , activation='relu',name = 'Hidden_Layer_4')(x)
output = Dense(num_digits , activation = 'softmax' , name = 'Outpur_layer')(x)

In [19]:
model = Model(inp,output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_4 (Dense)      (None, 200)               20200     
                                                                 
 Outpur_layer (Dense)        (None, 10)                2010      
                                                                 
Total params: 297910 (1.14 MB)
Trainable params: 297910 (1.14

In [20]:
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(learning_rate = learning_rate)

In [21]:
model.compile(loss = 'categorical_crossentropy',
              optimizer='sgd',
              metrics= ['accuracy'])

In [22]:
history1 = model.fit(x_train,y_train,
                    batch_size = batch_size,
                    epochs= training_epochs,
                    verbose= 2,
                    validation_data= (x_cv,y_cv))

Epoch 1/20
480/480 - 5s - loss: 1.5588 - accuracy: 0.5671 - val_loss: 0.6451 - val_accuracy: 0.8202 - 5s/epoch - 10ms/step
Epoch 2/20
480/480 - 3s - loss: 0.4726 - accuracy: 0.8676 - val_loss: 0.3712 - val_accuracy: 0.8923 - 3s/epoch - 7ms/step
Epoch 3/20
480/480 - 3s - loss: 0.3398 - accuracy: 0.9025 - val_loss: 0.2984 - val_accuracy: 0.9115 - 3s/epoch - 7ms/step
Epoch 4/20
480/480 - 3s - loss: 0.2897 - accuracy: 0.9169 - val_loss: 0.2636 - val_accuracy: 0.9223 - 3s/epoch - 7ms/step
Epoch 5/20
480/480 - 3s - loss: 0.2575 - accuracy: 0.9258 - val_loss: 0.2374 - val_accuracy: 0.9297 - 3s/epoch - 7ms/step
Epoch 6/20
480/480 - 3s - loss: 0.2330 - accuracy: 0.9337 - val_loss: 0.2138 - val_accuracy: 0.9355 - 3s/epoch - 7ms/step
Epoch 7/20
480/480 - 3s - loss: 0.2119 - accuracy: 0.9391 - val_loss: 0.2003 - val_accuracy: 0.9390 - 3s/epoch - 7ms/step
Epoch 8/20
480/480 - 3s - loss: 0.1951 - accuracy: 0.9443 - val_loss: 0.1861 - val_accuracy: 0.9440 - 3s/epoch - 7ms/step
Epoch 9/20
480/480 - 3s

In [23]:
# Model2

inp = Input(shape=(784,))
x =  Dense(n_hiddien_1 , activation='relu',name = 'Hidden_Layer_1')(inp)
x =  Dense(n_hiddien_2 , activation='relu',name = 'Hidden_Layer_2')(x)
x =  Dense(n_hiddien_3 , activation='relu',name = 'Hidden_Layer_3')(x)
x =  Dense(n_hiddien_4 , activation='relu',name = 'Hidden_Layer_4')(x)
output = Dense(num_digits , activation = 'softmax' , name = 'Outpur_layer')(x)

adam = keras.optimizers.Adam(learning_rate = learning_rate)
model2 = Model(inp,output)

model2.compile(loss = 'categorical_crossentropy',
              optimizer='adam',
              metrics= ['accuracy'])

In [24]:
history2 = model2.fit(x_train,y_train,
                    batch_size = batch_size,
                    epochs= training_epochs,
                    verbose= 2,
                    validation_data= (x_cv,y_cv))

Epoch 1/20
480/480 - 6s - loss: 0.2925 - accuracy: 0.9130 - val_loss: 0.1287 - val_accuracy: 0.9609 - 6s/epoch - 13ms/step
Epoch 2/20
480/480 - 4s - loss: 0.1123 - accuracy: 0.9659 - val_loss: 0.0918 - val_accuracy: 0.9714 - 4s/epoch - 8ms/step
Epoch 3/20
480/480 - 4s - loss: 0.0734 - accuracy: 0.9770 - val_loss: 0.0880 - val_accuracy: 0.9735 - 4s/epoch - 8ms/step
Epoch 4/20
480/480 - 4s - loss: 0.0550 - accuracy: 0.9825 - val_loss: 0.0857 - val_accuracy: 0.9732 - 4s/epoch - 8ms/step
Epoch 5/20
480/480 - 4s - loss: 0.0430 - accuracy: 0.9862 - val_loss: 0.0844 - val_accuracy: 0.9747 - 4s/epoch - 8ms/step
Epoch 6/20
480/480 - 4s - loss: 0.0348 - accuracy: 0.9888 - val_loss: 0.0868 - val_accuracy: 0.9781 - 4s/epoch - 8ms/step
Epoch 7/20
480/480 - 4s - loss: 0.0294 - accuracy: 0.9906 - val_loss: 0.0859 - val_accuracy: 0.9773 - 4s/epoch - 8ms/step
Epoch 8/20
480/480 - 4s - loss: 0.0258 - accuracy: 0.9916 - val_loss: 0.0887 - val_accuracy: 0.9781 - 4s/epoch - 8ms/step
Epoch 9/20
480/480 - 3s

In [25]:
# Model2a

inp = Input(shape=(784,))
x =  Dense(n_hiddien_1 , activation='relu',name = 'Hidden_Layer_1')(inp)
x =  Dense(n_hiddien_2 , activation='relu',name = 'Hidden_Layer_2')(x)
x =  Dense(n_hiddien_3 , activation='relu',name = 'Hidden_Layer_3')(x)
x =  Dense(n_hiddien_4 , activation='relu',name = 'Hidden_Layer_4')(x)
output = Dense(num_digits , activation = 'softmax' , name = 'Outpur_layer')(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(learning_rate = learning_rate)
model2a = Model(inp,output)

model2a.compile(loss = 'categorical_crossentropy',
              optimizer='adam',
              metrics= ['accuracy'])

In [26]:
history2a = model2a.fit(x_train,y_train,
                    batch_size = batch_size,
                    epochs= training_epochs,
                    verbose= 2,
                    validation_data= (x_cv,y_cv))

Epoch 1/20
480/480 - 6s - loss: 0.2896 - accuracy: 0.9141 - val_loss: 0.1418 - val_accuracy: 0.9580 - 6s/epoch - 13ms/step
Epoch 2/20
480/480 - 4s - loss: 0.1119 - accuracy: 0.9657 - val_loss: 0.0899 - val_accuracy: 0.9714 - 4s/epoch - 8ms/step
Epoch 3/20
480/480 - 4s - loss: 0.0739 - accuracy: 0.9773 - val_loss: 0.0972 - val_accuracy: 0.9703 - 4s/epoch - 8ms/step
Epoch 4/20
480/480 - 4s - loss: 0.0559 - accuracy: 0.9825 - val_loss: 0.0748 - val_accuracy: 0.9781 - 4s/epoch - 8ms/step
Epoch 5/20
480/480 - 4s - loss: 0.0446 - accuracy: 0.9851 - val_loss: 0.0721 - val_accuracy: 0.9797 - 4s/epoch - 8ms/step
Epoch 6/20
480/480 - 4s - loss: 0.0336 - accuracy: 0.9893 - val_loss: 0.0907 - val_accuracy: 0.9761 - 4s/epoch - 8ms/step
Epoch 7/20
480/480 - 4s - loss: 0.0334 - accuracy: 0.9895 - val_loss: 0.0901 - val_accuracy: 0.9762 - 4s/epoch - 8ms/step
Epoch 8/20
480/480 - 4s - loss: 0.0258 - accuracy: 0.9914 - val_loss: 0.0799 - val_accuracy: 0.9804 - 4s/epoch - 8ms/step
Epoch 9/20
480/480 - 4s

In [27]:

# Model2b

inp = Input(shape=(784,))
x =  Dense(n_hiddien_1 , activation='relu',name = 'Hidden_Layer_1')(inp)
x =  Dense(n_hiddien_2 , activation='relu',name = 'Hidden_Layer_2')(x)
x =  Dense(n_hiddien_3 , activation='relu',name = 'Hidden_Layer_3')(x)
x =  Dense(n_hiddien_4 , activation='relu',name = 'Hidden_Layer_4')(x)
output = Dense(num_digits , activation = 'softmax' , name = 'Outpur_layer')(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(learning_rate = learning_rate)
model2b = Model(inp,output)

model2b.compile(loss = 'categorical_crossentropy',
              optimizer='adam',
              metrics= ['accuracy'])

In [28]:
history2b = model2b.fit(x_train,y_train,
                    batch_size = batch_size,
                    epochs= training_epochs,
                    verbose= 2,
                    validation_data= (x_cv,y_cv))

Epoch 1/20
480/480 - 6s - loss: 0.2947 - accuracy: 0.9121 - val_loss: 0.1392 - val_accuracy: 0.9572 - 6s/epoch - 13ms/step
Epoch 2/20
480/480 - 4s - loss: 0.1125 - accuracy: 0.9656 - val_loss: 0.1076 - val_accuracy: 0.9669 - 4s/epoch - 8ms/step
Epoch 3/20
480/480 - 4s - loss: 0.0752 - accuracy: 0.9769 - val_loss: 0.0898 - val_accuracy: 0.9718 - 4s/epoch - 8ms/step
Epoch 4/20
480/480 - 4s - loss: 0.0551 - accuracy: 0.9823 - val_loss: 0.0826 - val_accuracy: 0.9747 - 4s/epoch - 8ms/step
Epoch 5/20
480/480 - 4s - loss: 0.0441 - accuracy: 0.9856 - val_loss: 0.0868 - val_accuracy: 0.9753 - 4s/epoch - 8ms/step
Epoch 6/20
480/480 - 4s - loss: 0.0354 - accuracy: 0.9884 - val_loss: 0.0906 - val_accuracy: 0.9751 - 4s/epoch - 8ms/step
Epoch 7/20
480/480 - 4s - loss: 0.0279 - accuracy: 0.9909 - val_loss: 0.0752 - val_accuracy: 0.9803 - 4s/epoch - 8ms/step
Epoch 8/20
480/480 - 4s - loss: 0.0278 - accuracy: 0.9910 - val_loss: 0.0863 - val_accuracy: 0.9774 - 4s/epoch - 8ms/step
Epoch 9/20
480/480 - 4s

In [29]:
# Model3

In [30]:
n_input = 784
n_hiddien_1 = 300
n_hiddien_2 = 100
n_hiddien_3 = 100
n_hiddien_4 = 100
n_hiddien_5 = 200
num_digits = 10

In [31]:
# Model3

inp = Input(shape=(784,))
x =  Dense(n_hiddien_1 , activation='relu',name = 'Hidden_Layer_1')(inp)
x =  Dense(n_hiddien_2 , activation='relu',name = 'Hidden_Layer_2')(x)
x =  Dense(n_hiddien_3 , activation='relu',name = 'Hidden_Layer_3')(x)
x =  Dense(n_hiddien_4 , activation='relu',name = 'Hidden_Layer_4')(x)
x =  Dense(n_hiddien_5 , activation='relu',name = 'Hidden_Layer_5')(x)
output = Dense(num_digits , activation = 'softmax' , name = 'Outpur_layer')(x)

In [32]:
model3 = Model(inp,output)
model3.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_4 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_5 (Dense)      (None, 200)               20200     
                                                                 
 Outpur_layer (Dense)        (None, 10)                2010

In [33]:
adam = keras.optimizers.Adam(learning_rate = 0.01)

model3.compile(loss = 'categorical_crossentropy',
              optimizer='adam',
              metrics= ['accuracy'])

In [34]:
history3 = model3.fit(x_train,y_train,
                    batch_size = batch_size,
                    epochs= training_epochs,
                    validation_data= (x_cv,y_cv))

Epoch 1/20
480/480 [==============================] - 7s 10ms/step - loss: 0.2977 - accuracy: 0.9115 - val_loss: 0.1322 - val_accuracy: 0.9598
Epoch 2/20
480/480 [==============================] - 4s 9ms/step - loss: 0.1101 - accuracy: 0.9665 - val_loss: 0.0950 - val_accuracy: 0.9694
Epoch 3/20
480/480 [==============================] - 5s 9ms/step - loss: 0.0779 - accuracy: 0.9753 - val_loss: 0.1002 - val_accuracy: 0.9685
Epoch 4/20
480/480 [==============================] - 4s 9ms/step - loss: 0.0570 - accuracy: 0.9818 - val_loss: 0.0810 - val_accuracy: 0.9772
Epoch 5/20
480/480 [==============================] - 4s 9ms/step - loss: 0.0437 - accuracy: 0.9859 - val_loss: 0.0918 - val_accuracy: 0.9729
Epoch 6/20
480/480 [==============================] - 4s 9ms/step - loss: 0.0362 - accuracy: 0.9884 - val_loss: 0.0842 - val_accuracy: 0.9765
Epoch 7/20
480/480 [==============================] - 4s 9ms/step - loss: 0.0363 - accuracy: 0.9885 - val_loss: 0.0803 - val_accuracy: 0.9778
Epoch

In [36]:
n_input = 784
n_hiddien_1 = 300
n_hiddien_2 = 100
n_hiddien_3 = 100
n_hiddien_4 = 200
num_digits = 10

In [37]:
# Model4

inp = Input(shape=(784,))
x =  Dense(n_hiddien_1 , activation='relu',name = 'Hidden_Layer_1')(inp)
x =  Dropout(0.3)(x)
x =  Dense(n_hiddien_2 , activation='relu',name = 'Hidden_Layer_2')(x)
x =  Dropout(0.3)(x)
x =  Dense(n_hiddien_3 , activation='relu',name = 'Hidden_Layer_3')(x)
x =  Dropout(0.3)(x)
x =  Dense(n_hiddien_4 , activation='relu',name = 'Hidden_Layer_4')(x)
output = Dense(num_digits , activation = 'softmax' , name = 'Outpur_layer')(x)

In [38]:
model4 = Model(inp,output)
model4.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0   

In [39]:
model4.compile(loss = 'categorical_crossentropy',
              optimizer='adam',
              metrics= ['accuracy'])

In [40]:
history = model4.fit(x_train,y_train,
                    batch_size = batch_size,
                    epochs= training_epochs,
                    validation_data= (x_cv,y_cv))

Epoch 1/20
480/480 [==============================] - 7s 11ms/step - loss: 0.4990 - accuracy: 0.8451 - val_loss: 0.1601 - val_accuracy: 0.9510
Epoch 2/20
480/480 [==============================] - 5s 10ms/step - loss: 0.2066 - accuracy: 0.9410 - val_loss: 0.1178 - val_accuracy: 0.9649
Epoch 3/20
480/480 [==============================] - 5s 10ms/step - loss: 0.1595 - accuracy: 0.9547 - val_loss: 0.0892 - val_accuracy: 0.9731
Epoch 4/20
480/480 [==============================] - 5s 10ms/step - loss: 0.1307 - accuracy: 0.9624 - val_loss: 0.0814 - val_accuracy: 0.9751
Epoch 5/20
480/480 [==============================] - 5s 10ms/step - loss: 0.1172 - accuracy: 0.9658 - val_loss: 0.0854 - val_accuracy: 0.9744
Epoch 6/20
480/480 [==============================] - 5s 10ms/step - loss: 0.1030 - accuracy: 0.9693 - val_loss: 0.0762 - val_accuracy: 0.9784
Epoch 7/20
480/480 [==============================] - 5s 10ms/step - loss: 0.0921 - accuracy: 0.9727 - val_loss: 0.0790 - val_accuracy: 0.9783

In [41]:
test_pred = pd.DataFrame(model4.predict(x_test,batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns= {0:'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

50/50 [==============================] - 0s 5ms/step


,ImageId,Label
0,1,7
1,2,2
2,3,1
3,4,0
4,5,4


In [42]:
test_pred.to_csv('mnist_submissio', index = False)